# Summarization on Dialog Data

Summarization creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. Summarization can be:

- **Extractive**: extract the most relevant information from a document.
- **Abstractive**: generate new text that captures the most relevant information.

### Imports

In [ ]:
!pip install transformers   
!pip install datasets
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

## Load DialogSum dataset

Start by loading the dialogsum from 🤗 Datasets library:

In [ ]:
from datasets import load_dataset
dialogsum = load_dataset("knkarthick/dialogsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-caf2f3e75d9073aa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dialogsum.shape

{'train': (12460, 4), 'test': (1500, 4), 'validation': (500, 4)}

There are two fields that you'll want to use:

- `dialog`: the text of the dialogue which'll be the input to the model.
- `summary`: a condensed version of `text` which'll be the model target.

Then take a look at an example:

In [ ]:
print(dialogsum["train"][0]['dialogue'])
print("-"*150)
print(dialogsum["train"][0]['summary'])

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.
--------------------------------------------------------------------

## Preprocess

The next step is to load a T5 tokenizer to process `dialog` and `summary`:

In [ ]:
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=64, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_dialogsum= dialogsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Now, the same dialog we saw in the previous output, is tokenized. 

In [ ]:
print("-"*40 + " DIALOG " + "-"*40)
print(tokenized_dialogsum['train'][0]['dialogue'])
print("-"*40 + " SUMMARY " + "-"*40)
print("\n"+tokenized_dialogsum['train'][0]['summary'])
print("-"*40 + " DIALOG TOKENS " + "-"*40)
print("\n"+ str(tokenized_dialogsum['train'][0]['input_ids']))
# print("-"*40 + " ATTENTION MASK " + "-"*40)
# print("\n"+ str(tokenized_dialogsum['train'][0]['attention_mask']))
print("-"*40 + " SUMMARY TOKENS " + "-"*40)
print("\n"+ str(tokenized_dialogsum['train'][0]['labels']))

---------------------------------------- DIALOG ----------------------------------------
#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#

In [ ]:
a = [21603, 10, 1713, 345, 13515, 536, 4663, 10, 2018, 6, 1363, 5, 3931, 5, 27, 31, 51, 7582, 12833, 77, 7, 5, 1615, 33, 25, 270, 469, 58, 1713, 345, 13515, 357, 4663, 10, 27, 435, 34, 133, 36, 3, 9, 207, 800, 12, 129, 3, 9, 691, 18, 413, 5, 1713, 345, 13515, 536, 4663, 10, 2163, 6, 168, 6, 25, 43, 29, 31, 17, 141, 80, 21, 305, 203, 5, 148, 225, 43, 80, 334, 215, 5, 1713, 345, 13515, 357, 4663, 10, 27, 214, 5, 27, 2320, 38, 307, 38, 132, 19, 1327, 1786, 6, 572, 281, 217, 8, 2472, 58, 1713, 345, 13515, 536, 4663, 10, 1548, 6, 8, 200, 194, 12, 1792, 2261, 21154, 19, 12, 253, 91, 81, 135, 778, 5, 264, 653, 12, 369, 44, 709, 728, 3, 9, 215, 21, 39, 293, 207, 5, 1713, 345, 13515, 357, 4663, 10, 8872, 5, 1713, 345, 13515, 536, 4663, 10, 1563, 140, 217, 270, 5, 696, 2053, 11, 11581, 320, 1399, 5, 2321, 3, 9, 1659, 6522, 6, 754, 5, 531, 25, 7269, 6, 1363, 5, 3931, 58, 1713, 345, 13515, 357, 4663, 10, 2163, 5, 1713, 345, 13515, 536, 4663, 10, 14627, 53, 19, 8, 1374, 1137, 13, 5084, 1874, 11, 842, 1994, 6, 25, 214, 5, 148, 310, 225, 10399, 5, 1713, 345, 13515, 357, 4663, 10, 27, 31, 162, 1971, 3986, 13, 648, 6, 68, 27, 131, 54, 31, 17, 1727, 12, 4583, 8, 7386, 5, 1713, 345, 13515, 536, 4663, 10, 1548, 6, 62, 43, 2287, 11, 128, 11208, 24, 429, 199, 5, 27, 31, 195, 428, 25, 72, 251, 274, 25, 1175, 5, 1713, 345, 13515, 357, 4663, 10, 8872, 6, 2049, 2472, 5, 1]
print(len(a))
b = [1363, 5, 3931, 31, 7, 652, 3, 9, 691, 18, 413, 6, 11, 7582, 12833, 77, 7, 7786, 7, 376, 12, 43, 80, 334, 215, 5, 12833, 77, 7, 31, 195, 428, 128, 251, 81, 70, 2287, 11, 11208, 12, 199, 1363, 5, 3931, 10399, 10257, 5, 1]
print(len(b))

286
48


Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate
rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

**Save your model in a drive location by editing the 'output_dir' argument**

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Models/DialogSum_1",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dialogsum["train"],
    eval_dataset=tokenized_dialogsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.524200,1.508175,0.366000,0.131300,0.304800,0.304700,18.858000
2,1.325900,1.480859,0.366500,0.133400,0.307500,0.307400,18.840000


TrainOutput(global_step=1558, training_loss=1.4045769589856285, metrics={'train_runtime': 562.8117, 'train_samples_per_second': 44.278, 'train_steps_per_second': 2.768, 'total_flos': 2830227572785152.0, 'train_loss': 1.4045769589856285, 'epoch': 2.0})

**[Optional]** Once training is completed, you can share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
type(model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [ ]:
# trainer.push_to_hub()

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to summarize. For T5, you need to prefix your input depending on the task you're working on. For summarization you should prefix your input as shown below:

In [ ]:
for key in dialogsum["validation"][2]:
  print(key.upper() + "\n", dialogsum["validation"][2][key])

ID
 dev_2
DIALOGUE
 #Person1#: I need to stop eating such unhealthy foods.
#Person2#: I know what you mean. I've started eating better myself.
#Person1#: What foods do you eat now?
#Person2#: I tend to stick to fruits, vegetables, and chicken.
#Person1#: Those are the only things you eat?
#Person2#: That's basically what I eat.
#Person1#: Why aren't you eating anything else?
#Person2#: Well, fruits and vegetables are very healthy.
#Person1#: And the chicken?
#Person2#: It's really healthy to eat when you bake it.
#Person1#: I guess that does sound a lot healthier.
SUMMARY
 #Person1# plans to stop eating unhealthy foods, and #Person2# shares #Person2#'s healthy recipe with #Person1#.
TOPIC
 healthy foods


In [ ]:
text =  dialogsum["validation"][2]["dialogue"]
summary = dialogsum["validation"][2]["summary"]

In [ ]:
for sentences in text.split("#Person"):
  print("#Person" + sentences)

#Person
#Person1#: I need to stop eating such unhealthy foods.

#Person2#: I know what you mean. I've started eating better myself.

#Person1#: What foods do you eat now?

#Person2#: I tend to stick to fruits, vegetables, and chicken.

#Person1#: Those are the only things you eat?

#Person2#: That's basically what I eat.

#Person1#: Why aren't you eating anything else?

#Person2#: Well, fruits and vegetables are very healthy.

#Person1#: And the chicken?

#Person2#: It's really healthy to eat when you bake it.

#Person1#: I guess that does sound a lot healthier.


Actual Summary 


In [ ]:
summary

"#Person1# plans to stop eating unhealthy foods, and #Person2# shares #Person2#'s healthy recipe with #Person1#."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for summarization with your model, and pass your text to it:

**Take the latest checkpoint from the model you saved in your drive**

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/drive/MyDrive/Models/DialogSum_1/checkpoint-1500")
summarizer(text)

Your max_length is set to 200, but you input_length is only 177. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)


[{'summary_text': '#Person1# wants to stop eating unhealthy foods. #Person2# tends to stick to fruits, vegetables, and chicken.'}]

#### [OPTIONAL]
You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return the `input_ids` as PyTorch tensors:

Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to create the summarization. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.  

*Decode* the generated token ids back into text:

In [ ]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForSeq2SeqLM

# inputs = tokenizer(text, return_tensors="pt").input_ids
# model.to('cpu')
# model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/Models/ft1/checkpoint-1500')
# # Experimenting with temperatures
# outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=0, temperature=0.3)
# tokenizer.decode(outputs[0], skip_special_tokens=True)
# outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=0, temperature=0.5)
# tokenizer.decode(outputs[0], skip_special_tokens=True)